# **Packages installation :**

In [ ]:
!pip install swifter


     |████████████████████████████████| 634kB 11.9MB/s 
     |████████████████████████████████| 471kB 26.9MB/s 
     |████████████████████████████████| 542kB 27.1MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 23.1MB 1.7MB/s 
     |████████████████████████████████| 17.2MB 188kB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 2.9MB 38.5MB/s 
     |████████████████████████████████| 1.3MB 42.1MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 133kB 36.4MB/s 
     |████████████████████████████████| 204kB 53.6MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 143kB 48.2MB/s 
     |████████████████████████████████| 296kB 43.9MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
  Created wheel for swifter: filename=swifter-1.0.7-cp36-no

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Import :**
The swifter package is used to fasten our calculcation by parallelizing our processes that used the apply function from pandas

In [ ]:
import pandas as pd
import swifter
import numpy as np
import math



#**File paths :**

In [ ]:
# output_file_path path to store the D matrix that we named interest 
# and the splited result: interest train and intrest test
output_file_path = "/content/drive/MyDrive/M2 MLDS/Documents  et cours/Fact/Results/"
# zip_file_path path to the recommendation.zip file
zip_file_path = "/content/drive/MyDrive/'M2 MLDS'/'Documents  et cours'/Fact/recommendation.zip"
# zip_file_output path where the result of extraction of zip file is stored
zip_file_output = "/content/drive/MyDrive/'M2 MLDS'/'Documents  et cours'/Fact/Results/"

In [ ]:
# extract the zip file
!unzip $zip_file_path -d $zip_file_output


Archive:  /content/drive/MyDrive/M2 MLDS/Documents  et cours/Fact/recommendation.zip
replace /content/drive/MyDrive/M2 MLDS/Documents  et cours/Fact/Results/recommendation/asset_genres.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# **Functions :**
We summerized the process of extracting the D dataset (**intrest dataset**) through one single function name **process_rui** that take as input the 3 datasets : bookmarks, ratings and favorites.<br>
We optimized this processed by using swifter to parallelize each apply function and we choosed to use the merge function instead of iterating the whole files to avoid slowing down the whole process and further optimise our method.<br>
The whole process os done in less then 2 min.<br>
**Remark :** in our work we decided to conisder films with the date time value with 0 from the bookmarks dataset as not watched to make the whole process more interesting then just setting the whole thing to one.
 $$
  r_{ui} = w_{ui} + n_{ui} + f_{ui},
  $$

* where:
  * $m_{ui} = 1$ if $u$ watched $i$ and $m_{ui} = 0$ if not;
  * $n_{ui}$ is the rating of $u$ to $i$ found in *ratings.csv*;
  * $n_{ui} = 0$ if not found in *ratings.csv*;  
  * $f_{ui} = 5$ if $u$ marked $i$ as favorite;
  * $f_{ui} = 0$ if not found in 'favorites.csv';

In [ ]:
"""
- Input:
Bookmarks : dataset
ratings : dataset 
favorites : dataset

- Output :
interest : the D dataset whose rows are aligned with the rows of table 'bookmarks.csv'

"""
def process_rui(bookmarks, ratings, favorites):
  interest = bookmarks[["id_profile", "id_asset"]]
  print("extracting wui column")
  interest["wui"] = bookmarks["time"].swifter.apply(lambda x: int(x > 0))

  print("exrtacting nui column")
  interest["rating"] = bookmarks.merge(ratings[["score", "id_profile", "id_asset"]], how="left", on=["id_profile", "id_asset"])["score"]
  favorites["fui"] = 5

  print("extracting fui column")
  interest["fui"] = bookmarks.merge(favorites[["fui", "id_profile", "id_asset"]], how="left", on=["id_profile", "id_asset"])["fui"]
  interest["wui"] = interest["wui"].fillna(0)
  interest["fui"] = interest["fui"].fillna(0)
  interest["nui"] = interest["rating"].fillna(0)


  
  print("processing rui")
  interest["rui"] = interest.swifter.apply(lambda x : x["wui"] + x["nui"] + x["fui"], axis= 1)
  del interest["wui"]
  del interest["nui"]


  return interest

# **Generating interest Dataset (D) :**
We load the files and used the function **process_rui** described above

In [ ]:
import pandas as pd

print("loading dataframes")
print("bookmarks")
bookmarks = pd.read_csv("/content/recommendation/bookmarks.csv")
print("ratings")
ratings = pd.read_csv("/content/recommendation/ratings.csv")
print("favorites")
favorites = pd.read_csv("/content/recommendation/favorites.csv")

print("getting interrest dataframe")

interest = process_rui(bookmarks, ratings, favorites)

del bookmarks
del ratings
del favorites

loading dataframes
bookmarks
ratings
favorites
getting interrest dataframe
extracting wui column



exrtacting nui column
extracting fui column
processing rui


## **Saving interest dataset to csv :**

In [ ]:
#saving using pandas  
interest.to_csv(output_file_path + "interest.csv", index_col="index")

In [ ]:
#visualizing the D dataset interest
interest

# **preparing training and testing set :**
Extracting the train and test set from the interest dataset using the given indexes.

In [ ]:
#loading the train and test indexes
idx_train = np.load(output_file_path + "/recommendation/bookmarks_idx_train.npy")
idx_test =np.load(output_file_path + "/recommendation/bookmarks_idx_test.npy")
#Extracing the test and train datasets
interest_train = interest.iloc[idx_train,:]
interest_test = interest.iloc[idx_test,:]


In [ ]:
# deleting unused data to save memory
del interest
del idx_train
del idx_test

## **Visualizing Train and Test set**

In [ ]:
interest_train

,id_profile,id_asset,rating,fui,rui
index,,,,,
39963458,326534,202094,NaN,0.0,0.0
41808284,133318,5343689,NaN,0.0,0.0
37973227,1101174,86739,NaN,0.0,1.0
43528603,1264136,5349884,NaN,0.0,0.0
9434040,1093487,1547,NaN,0.0,0.0
...,...,...,...,...,...
28904140,919396,4994709,NaN,0.0,0.0
57027276,1166571,6262931,NaN,0.0,0.0
25845032,1491922,4822771,NaN,0.0,0.0


In [ ]:
interest_test

,id_profile,id_asset,rating,fui,rui
index,,,,,
33554432,1045347,24217,NaN,0.0,0.0
67108865,1680166,6639497,NaN,0.0,0.0
3,10,24250,NaN,0.0,0.0
5,10,24444,NaN,0.0,0.0
6,10,24499,NaN,0.0,0.0
...,...,...,...,...,...
67108853,1029964,6788251,NaN,0.0,0.0
67108854,629630,6643818,NaN,0.0,0.0
33554426,1475141,5224553,NaN,0.0,0.0


## **Completing Training set with test set**
Some users and assets contained in the testing set, aren't represented in the training set, our models need atleast one occurence of each so that they would have weights, because on Baseline  Estimate and SVD++ the weights are directly linked to the specific users and assets (Bu, Bi, Qi, Pu, Yj).

### 1. **Completing missing assets in training set :**


In [ ]:
# getting the unique assets from the train and test sets
unique_asset_train = interest_train.id_asset.unique()
unique_asset_test = interest_test.id_asset.unique()

# the setdiff1d(A,B) function return the element that exist in A and do not exit in B 
print(len(unique_asset_train))
print(len(unique_asset_test))
#we can see that there is 118920 assets not present in the train set
print(len(np.setdiff1d(unique_asset_test, unique_asset_train)))


1774065
1464899
118920


In [ ]:
# we decided to add all the rows containing the 118920 assets to the train set and also removing them from the test set. 
interest_train=interest_train.append(interest_test[interest_test.id_asset.isin(np.setdiff1d(unique_asset_test, unique_asset_train))])
interest_test=interest_test[~interest_test.id_asset.isin(np.setdiff1d(unique_asset_test, unique_asset_train))]
del unique_asset_train
del unique_asset_test

### **2. Completing missing users in training set :**

In [ ]:
# getting the unique assets from the train and test sets
unique_user_train = interest_train.id_profile.unique()
unique_user_test = interest_test.id_profile.unique()
print(len(unique_user_train))
print(len(unique_user_test))

1171799
1045155


In [ ]:
# we decided to add all the rows containing the users to the train set and also removing them from the test set. 
interest_train=interest_train.append(interest_test[interest_test.id_profile.isin(np.setdiff1d(unique_user_test, unique_user_train))])
interest_test=interest_test[~interest_test.id_profile.isin(np.setdiff1d(unique_user_test, unique_user_train))]
del unique_user_train
del unique_user_test

After preprocessing, there are no users nor assets in testing set that aren't in training set

In [ ]:
unique_user_train = interest_train.id_profile.unique()
unique_user_test = interest_test.id_profile.unique()
print(len(np.setdiff1d(unique_user_test, unique_user_train)))
del unique_user_train
del unique_user_test

0


In [ ]:
unique_asset_train = interest_train.id_asset.unique()
unique_asset_test = interest_test.id_asset.unique()
print(len(np.setdiff1d(unique_asset_test, unique_asset_train)))
del unique_asset_train
del unique_asset_test

0


## **removing duplicates from training and testing set :**

In [ ]:
interest_train.drop_duplicates(subset=["id_profile", "id_asset"], inplace=True)
interest_test.drop_duplicates(subset=["id_profile", "id_asset"], inplace=True)

## **Saving results :**

In [ ]:
interest_train.to_csv(output_file_path  + "interest_train.csv", index_label = "index")
interest_test.to_csv(output_file_path + "interest_test.csv", index_label = "index")   